In [1]:
import os;
import sys;
sys.path.append("venv")

In [2]:
import pandas as pd
pd.read_parquet('../WSE_marketdata.parquet', engine='pyarrow')

,timestamp,symbol,bid_price,bid_size,ask_price,ask_size,market_period,trade_price,trade_volume,type
0,2024-01-22 05:04:15,NVA,18.0000,419.0,18.700,453.0,,NaN,NaN,Quote
1,2024-01-22 05:04:15,BSA,0.2220,1800.0,0.243,2500.0,,NaN,NaN,Quote
2,2024-01-22 05:04:15,NXG,2.6000,200.0,2.670,190.0,,NaN,NaN,Quote
3,2024-01-22 05:04:15,DTX,0.1275,6724.0,0.140,16228.0,,NaN,NaN,Quote
4,2024-01-22 05:04:15,PXM,4.0000,1100.0,4.020,1000.0,,NaN,NaN,Quote
...,...,...,...,...,...,...,...,...,...,...
645580,2024-01-22 17:05:00,ULG,9.9000,167.0,9.900,196.0,LAMO,NaN,NaN,Quote
645581,2024-01-22 17:05:00,ALE,30.2450,500.0,30.245,10310.0,LAMO,NaN,NaN,Quote
645582,2024-01-22 17:05:00,SVE,4.3200,1300.0,4.320,311.0,LAMO,NaN,NaN,Quote
645583,2024-01-22 17:05:21,ACT,NaN,NaN,NaN,NaN,None,13.1,26500.0,Trade


In [3]:
pd.read_parquet('../WSE_symbology.parquet', engine='pyarrow')

,symbol
0,QNT
1,SDG
2,ENI
3,EKS
4,GIF
...,...
782,CWA
783,MDP
784,ORG
785,MFO


In [11]:
from regionConfig.WSE import WSE
# Spread sizes

import aggregator
import filters


class SpreadByTickSizeAggregate(aggregator.Aggregate):
    def __init__(self, region, symbol):
        self.last_bid = 0.0
        self.last_ask = 0.0
        self.region = region
        self.tick_series = []

    def on_quote(self, quote):
        self.spread = quote.ask_price - quote.bid_price

    def on_trade(self, trade):
        tick_size = self.region.get_tick_size(trade.trade_price, trade.symbol)
        if tick_size != 0:
            self.tick_series.append(self.spread / tick_size)

    def compute_slice(self, slice):
        if len(self.tick_series) == 0:
            return 0

        value = sum(self.tick_series) / len(self.tick_series)
        return value



agg = aggregator.Aggregator("../WSE_marketdata.parquet","../WSE_symbology.parquet", WSE(), filters.SymbolFilter("DTR"))
agg.registerAggregate(SpreadByTickSizeAggregate)
aggs = agg.run()
aggs

,symbol,slice,aggregate,value
0,DTR,09:00:00,SpreadByTickSize,0
1,DTR,09:05:00,SpreadByTickSize,0


In [13]:
from regionConfig.WSE import WSE
# Spread sizes

import aggregator
import filters

class TradeCountAggregate(aggregator.Aggregate):
    def __init__(self, region, symbol):
        self.trade_count = 0

    def on_quote(self, quote):
        pass

    def on_trade(self, trade):
        self.trade_count+=1

    def compute_slice(self, slice):
        val = self.trade_count
        self.trade_count = 0 
        return val

agg = aggregator.Aggregator("../WSE_marketdata.parquet","../WSE_symbology.parquet", WSE())
agg.registerAggregate(TradeCountAggregate)
trade_count_aggs = agg.run()

In [25]:
import pandas as pd
import datetime
first_slice_time = datetime.time(9,0,0)
trade_counts = trade_count_aggs.groupby('symbol')["value"].sum()
pd.DataFrame(trade_counts).query('value == 0')

,value
symbol,
7FT,0
AAS,0
ABK,0
ADS,0
AFC,0
...,...
XBS,0
YAN,0
YBS,0


In [9]:
import datetime
time = datetime.time(15,50,00)
aggs.query('slice == @time and value == 0')
#aggs.query('symbol == "PKO"')

,symbol,slice,aggregate,value


In [14]:
import sys
import aggregator
import filters


class volumeAggregate(aggregator.Aggregate):
    def __init__(self):
        self.volume = 0

    def on_quote(self, quote):
        pass

    def on_trade(self, trade):
        self.volume += trade.trade_volume

    def compute_slice(self, slice):
        ret = self.volume
        self.volume = 0
        return ret


agg = aggregator.Aggregator("../WSE_marketdata.parquet","../WSE_symbology.parquet", filters.SymbolFilter('PKO'))
agg.registerAggregate(volumeAggregate)
agg.run()

,symbol,slice,aggregate,value
